In [9]:
import random
import covalent as ct
import pennylane as qml

# Define a Qiskit Runtime executor.
# qiskit_sampler = ct.executor.QiskitExecutor(
#         device="sampler",
#         backend="ibmq_qasm_simulator",
#         single_job=False,
# )
qiskit_qasm = ct.executor.QiskitExecutor(
    device="sampler",
    backend="ibmq_qasm_simulator",
    ibmqx_token="b6719d7786262f6ae9e16a28a67c5484a2541c988f1912d755185f840c2dcbce938a1ca695dc87f0dc0dd943529919dc003ca93fae479e741540e3277d3904b4",  # required if not in config file
    single_job=False,
)


# Create a QElectron that executes on Qiskit Runtime.
@ct.qelectron(executors=qiskit_qasm)
@qml.qnode(qml.device("default.qubit", wires=2, shots=1024))
def circuit(x):
    qml.RX(x, wires=0)
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliY(0))


# Create Electrons for a simple workflow.
@ct.electron
def generate_rand_input():
    return random.uniform(0, 3.14159)

@ct.electron
def run_qiskit_experiment(x):
    # Workflow task that uses the `circuit` QElectron.
    return circuit(x)

@ct.lattice
def workflow():
    results = []
        
    # Six independent experiments.
    for _ in range(6):
        x = generate_rand_input()
        results.append(run_qiskit_experiment(x))

    return results

# Dispatch workflow.
dispatch_id = ct.dispatch(workflow)()
print(dispatch_id)
results = ct.get_result(dispatch_id, wait=True).result

023a6d1f-058a-46cf-b0d3-a0bdf45bd8cb


In [8]:
results